In [ ]:
!pip install hummingbird_ml[extra,onnx]

In [1]:
import torch
import numpy as np
import lightgbm as lgb

import onnxruntime as ort
from onnxmltools.convert import convert_lightgbm
from onnxconverter_common.data_types import FloatTensorType

from hummingbird.ml import convert
from hummingbird.ml import constants

# Create some random data for binary classification.
num_classes = 2
X = np.array(np.random.rand(10000, 28), dtype=np.float32)
y = np.random.randint(num_classes, size=10000)

In [2]:
# Create and train a model (LightGBM in this case).
model = lgb.LGBMClassifier()
model.fit(X, y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [3]:
# Use ONNXMLTOOLS to convert the model to ONNXML.
initial_types = [("input", FloatTensorType([X.shape[0], X.shape[1]]))] # Define the inputs for the ONNX
onnx_ml_model = convert_lightgbm(
    model, initial_types=initial_types, target_opset=9
)

In [4]:
# Use Hummingbird to convert the ONNXML model to ONNX.
onnx_model = convert(onnx_ml_model, "onnx", X)

In [5]:
# Alternatively we can set the inital types using the extra_config parameters as in the ONNXMLTOOL converter.
extra_config = {}
extra_config[constants.ONNX_INITIAL_TYPES] = initial_types
onnx_model = convert(onnx_ml_model, "onnx", extra_config=extra_config)

In [6]:
# Get the predictions for the ONNX model
session = ort.InferenceSession(onnx_model.SerializeToString())
output_names = [session.get_outputs()[i].name for i in range(len(session.get_outputs()))]
inputs = {session.get_inputs()[0].name: X}

In [7]:
%%timeit -r 3

# Run the ONNX model on CPU 
session.run(output_names, inputs)

2.61 s ± 43.8 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
